In [5]:
from transformers import pipeline
import evaluate

metric = evaluate.load("sacrebleu")

2024-08-05 17:38:43.154364: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 17:38:43.154418: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 17:38:43.159733: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 17:38:43.837982: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from datasets import load_dataset
test_dataset = load_dataset('pandas', data_files='/home/j/Documents/Projects/MLotsawa/data/size-selection-data/100k-test.p')
test_dataset = test_dataset['train']['translation']
inputs = [elt['bo'] for elt in test_dataset]
references = [elt['en'] for elt in test_dataset]

bo = 'sangye ch dang tsok kyi chok nam la. changchub bardu dak ni kyab su chi. dak gi jin sok gyip snam kyi. dro la pen chir sangye drubpar shok.'
split = bo.split('.')

In [8]:
results = {}
for size in ['small', 'base', 'large']:
    results[size] = {}

    print(f'Checking {size} model. Generating predictions...')
    
    translator = pipeline("translation_bo_to_en", model=f'/home/j/Documents/Projects/MLotsawa/models/size-selection/{size}/checkpoint-375000', device=0, max_length=100)
    outputs = translator(inputs)

    print(f'Predictions generated. Computing score...')

    outputs = [elt['translation_text'] for elt in outputs]
    result = metric.compute(predictions=outputs, references=references)
    score = result['score']
    results[size]['bleu'] = score
    print(f'Score: {score}. Creating sample translation.')

    translation = translator(split)
    translation = [elt['translation_text'] for elt in translation]
    results[size]['translation'] = translation

    print(f'{size} model finished. Moving on.')


Checking small model. Generating predictions...
Predictions generated. Computing score...
Score: 3.742805542416307. Creating sample translation.
small model finished. Moving on.
Checking base model. Generating predictions...


KeyboardInterrupt: 

## Sample Translation 

sangye ch dang tsok kyi chok nam la

changchub bardu dak ni kyab su chi

dak gi jin sok gyip snam kyi

dro la pen chir sangye drubpar shok

***

In the Buddha the Dharma and the Supreme Assembly

I take refuge until I attain enlightenment

Through the merit of practising generosity and so on

May I attain buddhahood for the benefit of all beings